# Redes

Muchos temas apasionantes, como la visión por ordenador, el Internet de las cosas (IoT) y los bots de chat con IA, dominan los titulares. Esto hace que se olviden las tecnologías básicas tradicionales. Aunque es fantástico tener muchas capacidades nuevas al alcance de la mano, es importante darse cuenta de que estas tecnologías no serían posibles sin una base sólida de conectividad fiable y segura. El procesamiento de datos sólo es útil si los resultados se entregan de forma fiable y son accesibles a través de una red. Los contenedores son un método fantástico de despliegue de aplicaciones, pero proporcionan la mejor experiencia para los usuarios cuando están conectados en red.

Los servicios y características de red dentro de AWS son la columna vertebral de casi todos los grandes servicios que cubrimos en este libro. AWS tiene muchas características excelentes para que conectes lo que quieras, donde quieras y como quieras. Obtener una mejor comprensión de las redes le permitirá tener una mejor comprensión de la nube y por lo tanto estar más cómodo usándola.

### Definición de su red virtual privada en la nube mediante la creación de una VPC de Amazon

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0201.png" width="650">

In [ ]:
VPC_ID=$(aws ec2 create-vpc --cidr-block 10.10.0.0/16 \
    --tag-specifications 'ResourceType=vpc,Tags=[{Key=Name,Value=AWSCookbook201}]' \
    --output text --query Vpc.VpcId)

*Cuando se crea una VPC, la documentación indica que el mayor tamaño de bloque para los CIDR IPv4 de la VPC es una máscara de red /16 (65.536 direcciones IP). El más pequeño es una máscara de red /28 (16 direcciones IP).*

In [ ]:
# Utilice este comando para verificar que el estado de la VPC está disponible:
aws ec2 describe-vpcs --vpc-ids $VPC_ID

Hay dos razones importantes para seleccionar cuidadosamente los bloques CIDR para su VPC:

* Una vez que un bloque CIDR está asociado a una VPC, no puede ser modificado (aunque puede ser ampliado). Si desea modificar un bloque CIDR, deberá eliminarlo (y todos los recursos que contiene) y volver a crearlo.

* Si una VPC está conectada a otras redes mediante peering o pasarelas (por ejemplo, Tránsito y VPN), tener rangos de IP superpuestos causará problemas no deseados.

Puede añadir espacio IPv4 a la VPC utilizando el comando aws ec2 associate-vpc-cidr-block para especificar el espacio IPv4 adicional. Cuando el espacio IP escasea por el uso y el subaprovisionamiento, es bueno saber que no es necesario dedicar un gran bloque a una VPC, especialmente si no está seguro de que se utilizará todo.

In [ ]:
# Este es un ejemplo de asociación de un bloque CIDR IPv4 adicional a su VPC:
aws ec2 associate-vpc-cidr-block \
    --cidr-block 10.11.0.0/16 \
    --vpc-id $VPC_ID

Amazon VPC admite direcciones IPv4 e IPv6 y tiene cuotas de tamaño de bloque de CIDR diferentes para cada una de ellas. De forma predeterminada, todas las VPC y subredes deben tener bloques de CIDR IPv4. Esto no se puede cambiar. Si lo desea, puede asociar un bloque de CIDR IPv6 a su VPC especificando la opción --amazon-provided-ipv6-cidr-block. 

Este es un ejemplo de creación de una VPC con un bloque CIDR IPv6:

In [ ]:
aws ec2 create-vpc --cidr-block 10.10.0.0/16 \
    --amazon-provided-ipv6-cidr-block \
    --tag-specifications 'ResourceType=vpc,Tags=[{Key=Name,Value=AWSCookbook201-IPv6}]'

Una VPC es una construcción regional en AWS. Una región es un área geográfica, y las zonas de disponibilidad son centros de datos físicos que residen en una región. Las regiones abarcan todas las zonas de disponibilidad (AZ), que son grupos de centros de datos físicos aislados. El número de AZs por región varía, pero todas las regiones tienen al menos tres. Para obtener la información más actualizada sobre las regiones y las zonas de disponibilidad de AWS, consulte este artículo sobre ["Regiones y zonas de disponibilidad"](https://aws.amazon.com/es/about-aws/global-infrastructure/regions_az/).

Según la guía del usuario de la VPC, la cuota inicial de bloques CIDR IPv4 por VPC es de 5. Puede aumentarse a 50. El número permitido de bloques CIDR IPv6 por VPC es 1.

### Creación de un nivel de red con subredes y una tabla de rutas en una VPC

Cree una tabla de rutas dentro de su VPC. Cree dos subredes en zonas de disponibilidad separadas en una VPC. Asocie la tabla de rutas con las subredes.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0202.png" width="650">


In [ ]:
# Crear una tabla de rutas. Esto le permitirá crear rutas de tráfico personalizadas para las subredes asociadas:
ROUTE_TABLE_ID=$(aws ec2 create-route-table --vpc-id $VPC_ID \
    --tag-specifications \
'ResourceType=route-table,Tags=[{Key=Name,Value=AWSCookbook202}]' \
    --output text --query RouteTable.RouteTableId)

In [ ]:
# Cree dos subredes, una en cada AZ. Esto definirá los espacios de direcciones para que pueda crear recursos para su VPC:
SUBNET_ID_1=$(aws ec2 create-subnet --vpc-id $VPC_ID \
    --cidr-block 10.10.0.0/24 --availability-zone ${AWS_REGION}a \
    --tag-specifications \
    'ResourceType=subnet,Tags=[{Key=Name,Value=AWSCookbook202a}]' \
    --output text --query Subnet.SubnetId)

SUBNET_ID_2=$(aws ec2 create-subnet --vpc-id $VPC_ID \
    --cidr-block 10.10.1.0/24 --availability-zone ${AWS_REGION}b \
    --tag-specifications \
    'ResourceType=subnet,Tags=[{Key=Name,Value=AWSCookbook202b}]' \
    --output text --query Subnet.SubnetId)

En los comandos anteriores, el parámetro `--availability-zone` utiliza una variable de entorno para su región añadiendo caracteres a o b en minúsculas para indicar qué AZ lógica (por ejemplo, us-east-1a) debe aprovisionar cada subred. AWS afirma que estos nombres son aleatorios por cuenta para equilibrar los recursos entre las AZ.

Si utiliza varias cuentas de AWS y desea encontrar ID de zona de disponibilidad para una región que sean coherentes, ejecute este comando:

In [ ]:
aws ec2 describe-availability-zones --region $AWS_REGION

In [ ]:
# Asociar la tabla de rutas con las dos subredes:
aws ec2 associate-route-table \
    --route-table-id $ROUTE_TABLE_ID --subnet-id $SUBNET_ID_1

aws ec2 associate-route-table \
    --route-table-id $ROUTE_TABLE_ID --subnet-id $SUBNET_ID_2

In [ ]:
# Recupere la configuración de las subredes que creó y verifique que están en la misma VPC pero en diferentes AZs:
aws ec2 describe-subnets --subnet-ids $SUBNET_ID_1
aws ec2 describe-subnets --subnet-ids $SUBNET_ID_2

In [ ]:
# Valida que la tabla de rutas que has creado está asociada a las dos subredes:
aws ec2 describe-route-tables --route-table-ids $ROUTE_TABLE_ID

Al diseñar una estrategia de subred, debe elegir tamaños de subred que se ajusten a sus necesidades actuales y tengan en cuenta el crecimiento futuro de su aplicación. Las subredes se utilizan para la colocación de la interfaz de red elástica (ENI) para los recursos de AWS. Esto significa que una ENI concreta vive dentro de una única AZ.

AWS reserva las cuatro primeras y la última dirección IP del bloque CIDR de cada subred para características y funcionalidades cuando se crea una subred. Estas no están disponibles para su uso. Según la documentación, estas son las direcciones reservadas en el caso de su ejemplo:

* *.0* - Dirección de red.
* *.1* - Reservada por AWS para el router de la VPC.
* *.2* - Reservada por AWS para la dirección IP del servidor DNS. Siempre se establece el rango de red de la VPC más dos.
* *.3* - Reservada por AWS para uso futuro.
* *.255* - Dirección de difusión de la red. No se admite la difusión en una VPC.

Una subred tiene una tabla de rutas asociada a ella. Las tablas de rutas pueden estar asociadas a una o más subredes y dirigir el tráfico a un destino de su elección (más adelante se hablará de esto con las recetas de pasarela NAT, gateway de Internet y gateway de tránsito). Las entradas dentro de las tablas de rutas se llaman rutas y se definen como pares de Destinos y Objetivos. Al crear la tabla de rutas, se añadió automáticamente una ruta local por defecto que gestiona el tráfico intra-VPC. Tiene la posibilidad de crear rutas personalizadas que se ajusten a sus necesidades. Para obtener una lista completa de los destinos disponibles para utilizar dentro de las tablas de rutas, consulte este [documento de soporte](https://oreil.ly/oKVq1).

*Las ENIs reciben una dirección IP de un servidor DHCP administrado por AWS dentro de su VPC. El conjunto de opciones DHCP se configura automáticamente con valores predeterminados para asignar direcciones dentro de las subredes que usted defina.*


Cuando se crea una VPC en una región, es una buena práctica repartir las subredes entre las AZ de ese nivel de red. El número de AZs varía según la región, pero la mayoría tienen al menos tres. Un ejemplo de esto en la práctica sería que si tuvieras un tier público y un tier aislado repartidos en dos AZs, tendrías un total de cuatro subredes: 2 tiers × 2 subredes por tier (una por AZ).

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0203.png" width="600">

### Conexión de su VPC a Internet mediante un gateway de Internet

Crearás un gateway de internet y lo adjuntarás a tu VPC. A continuación, modificará la tabla de rutas asociada a la subred en la que se encuentra la instancia EC2. Añadirás una ruta que envíe el tráfico de las subredes a la gateway de Internet. Por último, crea una IP elástica (EIP) y asóciala a la instancia:

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0204.png" width="700">

In [ ]:
# Crear un gateway de Internet (IGW):
INET_GATEWAY_ID=$(aws ec2 create-internet-gateway \
    --tag-specifications \
    'ResourceType=internet-gateway,Tags=[{Key=Name,Value=AWSCookbook202}]' \
    --output text --query InternetGateway.InternetGatewayId)

In [ ]:
# Adjunte el gateway de Internet a la VPC existente:
aws ec2 attach-internet-gateway \
    --internet-gateway-id $INET_GATEWAY_ID --vpc-id $VPC_ID

In [ ]:
# En cada tabla de rutas de su VPC, cree una ruta que establezca el destino de la ruta por defecto a la gateway de Internet:
aws ec2 create-route --route-table-id $ROUTE_TABLE_ID_1 \
    --destination-cidr-block 0.0.0.0/0 --gateway-id $INET_GATEWAY_ID
&&
aws ec2 create-route --route-table-id $ROUTE_TABLE_ID_2 \
    --destination-cidr-block 0.0.0.0/0 --gateway-id $INET_GATEWAY_ID

In [ ]:
# Crear una EIP (dirección IP elástica):
ALLOCATION_ID=$(aws ec2 allocate-address --domain vpc \
    --output text --query AllocationId)

*AWS define una dirección IP elástica (EIP) como "una dirección IPv4 estática diseñada para la computación dinámica en la nube. Una dirección EIP se asigna a su cuenta de AWS y es suya hasta que la libere".*

In [ ]:
# Asocie el EIP con la instancia EC2 existente:
aws ec2 associate-address \
    --instance-id $INSTANCE_ID --allocation-id $ALLOCATION_ID